In [17]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

BASE_DIR = '../../../'

DATA_FP = BASE_DIR + '/data/processed/'
MAP_FP = DATA_FP + 'maps/'


In [18]:
def parse_json(jsonfile, otherfields=[]):
    items = json.load(open(jsonfile))
    locations = {}
    for item in items:
        if str(item['near_id']) not in locations.keys():
            d = {'count': 0}
            for field in otherfields:
                d[field] = []
            locations[str(item['near_id'])] = d
        locations[str(item['near_id'])]['count'] += 1
        for field in otherfields:
            locations[str(item['near_id'])][field].append(item[field])
    return items, locations


In [19]:
def is_inter(id):
    if len(str(id)) > 1 and str(id)[0:2] == '00':
        return False
    return True

In [115]:
crash_data, crashes = parse_json(DATA_FP + 'crash_joined.json', otherfields=['FIRST_EVENT_SUBTYPE'])

concern_data, concerns = parse_json(DATA_FP + 'concern_joined.json',
                                        otherfields=['REQUESTTYPE'])
print crashes['0013023']

{'count': 1, 'FIRST_EVENT_SUBTYPE': [u'NO INJURIES']}


In [149]:
"""                                                                                                                                                                 
Info about intersections vs non intersections and their crash rate                                                                                                  
"""

# Hard code counts in since they don't change (at least not for Boston)                                                                                             
# and it's much faster                                                                                                                                              
inter_count = 8574
non_inter_count = 17388

counts = {
    'inter': 0,
    'non_inter': 0,
    'no_match': 0,
    'inter_plus': 0,
    'non_inter_plus': 0,
    'ped_inter': 0,
    'ped_non_inter': 0,
}

for k, v in crashes.iteritems():
    crash_types = v['FIRST_EVENT_SUBTYPE']
    ped = False
    for t in crash_types:
        if 'PEDESTRIAN' in t:
            ped = True
    # Add ped crash info to crashes
    crashes[k]['ped'] = ped

    if str(k) == '':
        counts['no_match'] += 1
    elif is_inter(k):
        if int(v['count']) > 1:
            counts['inter_plus'] += 1
        if ped:
            counts['ped_inter'] += 1
        counts['inter'] += 1
    else:
        if int(v['count']) > 1:
            counts['non_inter_plus'] += 1
        if ped:
            counts['ped_non_inter'] += 1
        counts['non_inter'] += 1

print "Number of intersections:" + str(inter_count)
print "Number of non-intersections:" + str(non_inter_count)

print "Number of intersection segments with 1/more than 1 crash:" \
    + str(counts['inter']) + '/' + str(counts['inter_plus'])
print "Number of non-intersection segments with 1/more than 1 crash:" \
    + str(counts['non_inter']) + '/' + str(counts['non_inter_plus'])

total_percent = round(float(100 * (counts['inter'] + counts['non_inter']) / float(inter_count + non_inter_count)))
print "percent of all segments with crash:" + str(total_percent)
print 'inter counts:' + str(counts['inter'])
print 'non inter counts:' + str(counts['non_inter'])

print float(100 * counts['inter'] + counts['non_inter']) / float(inter_count + non_inter_count)

# Percentage of intersections/non-intersections                                                                                                                     
# that have at least one crash
inter_percent = round(float(100 * counts['inter'])/float(inter_count))
print "percent of intersections with crash:" + str(inter_percent)

non_inter_percent = round(100 * float(counts['non_inter'])/float(non_inter_count))
print "percent of non-intersections with crash:" + str(non_inter_percent)

print "percent of intersections with more than 1 crash:" + str(
    round(float(100 * counts['inter_plus'])/float(inter_count)))

print "percent of non-intersections with more than 1 crash:" + str(
    round(float(100 * counts['non_inter_plus'])/float(non_inter_count)))

ped_total = counts['ped_inter'] + counts['ped_non_inter']
ped_percent = round(float(100 * ped_total)/float(inter_count + non_inter_count))
print "percentage of total segments that had a pedestrian crash:" + str(ped_percent)

ped_inter_percent = round(float(100 * counts['ped_inter'])/float(inter_count))
print "percentage of intersections that had a pedestrian crash:" + str(ped_inter_percent)

ped_non_inter_percent = round(float(100 * counts['ped_non_inter'])/float(non_inter_count))
print "percentage of non-intersections that had a pedestrian crash:" + str(ped_non_inter_percent)



Number of intersections:8574
Number of non-intersections:17388
Number of intersection segments with 1/more than 1 crash:1877/914
Number of non-intersection segments with 1/more than 1 crash:1552/369
percent of all segments with crash:13.0
inter counts:1877
non inter counts:1552
7.28957707419
percent of intersections with crash:22.0
percent of non-intersections with crash:9.0
percent of intersections with more than 1 crash:11.0
percent of non-intersections with more than 1 crash:2.0
percentage of total segments that had a pedestrian crash:4.0
percentage of intersections that had a pedestrian crash:7.0
percentage of non-intersections that had a pedestrian crash:2.0


In [145]:
"""                                                                                                                                                                 
What percentage of intersections with concerns had crashes                                                                                                          
at varying counts of concerns?                                                                                                                                      
"""

matching = {}
for id, d in concerns.iteritems():
    if d['count'] not in matching.keys():
        matching[d['count']] = {
            'inter': [0, 0],
            'non_inter': [0, 0]
        }
    if is_inter(id):
        key = 'inter'
    else:
        key = 'non_inter'

    if id in crashes.keys():
        matching[d['count']][key][0] += 1
    else:
        matching[d['count']][key][1] += 1

sorted_matching = sorted(matching.items())

results = []
for key, value in sorted_matching:

    # Do the 1+,2+ stats as well as 1, 2                                                                                                                            
    # Still need to break it out by int/non-int                                                                                                                     
    counts = {
        'total': value['inter'][0] + value['inter'][1] + value['non_inter'][0] + value['non_inter'][1],
        'crashes': value['inter'][0] + value['non_inter'][0],
        'inters_total': value['inter'][0] + value['inter'][1],
        'inters_crashes': value['inter'][0],
        'non_inters_total': value['non_inter'][0] + value['non_inter'][1],
        'non_inters_crashes':  value['non_inter'][0],
    }
    # Add all the data for segments with more complaints than the current complaint # we're on
    for key2, value2 in sorted_matching:
        if key2 > key:
            counts['total'] += value2['inter'][0] + value2['inter'][1] + value2['non_inter'][0] + value2['non_inter'][1]
            counts['crashes'] += value2['inter'][0] + value2['non_inter'][0]
            counts['inters_total'] += value2['inter'][0] + value2['inter'][1]
            counts['inters_crashes'] += value2['inter'][0]
            counts['non_inters_total'] += value2['non_inter'][0] + value2['non_inter'][1]
            counts['non_inters_crashes'] += value2['non_inter'][0]
    results.append([
        key,
        round(100 * float(counts['crashes'])/float(counts['total'])), # total percent
        (100 * float(counts['crashes'])/float(counts['total']))/float(total_percent),
        counts['total'], # total count
        round(float(counts['inters_crashes'])/float(counts['inters_total']) * 100), # total percent at intersections
        (float(counts['inters_crashes'])/float(counts['inters_total']) * 100)/inter_percent,
        counts['inters_total'], # total # of intersections with this many or more complaints
        round((float(counts['non_inters_crashes'])/float(counts['non_inters_total']) * 100) if counts['non_inters_total'] else 0), # total percent at non-intersections
        ((float(counts['non_inters_crashes'])/float(counts['non_inters_total']) * 100) if counts['non_inters_total'] else 0)/non_inter_percent,
        counts['non_inters_total'] # total # of non-intersections with this many or more complaints
    ])

pd.DataFrame(results, columns=[
    '# of concerns',
    '% of segments w/ crash',
    '# of times more likely than average segment',
    'total # of segments with this many or more concerns',
    '% of intersections with this many or more concerns with a crash',
    '# of times more likely than average intersection',
    '# of intersections with this many concerns',
    '% of non-intersections with this many concerns with a crash',
    '# of times more likely than the average non-intersection',
    '# of non-intersections with this many concerns'
])

,# of concerns,% of segments w/ crash,# of times more likely than average segment,total # of segments with this many or more concerns,% of intersections with this many or more concerns with a crash,# of times more likely than average intersection,# of intersections with this many concerns,% of non-intersections with this many concerns with a crash,# of times more likely than the average non-intersection,# of non-intersections with this many concerns
0,1,29.0,4.188297,4458,43.0,1.969260,2082,17.0,1.893939,2376
1,2,42.0,6.030151,1592,58.0,2.634059,868,23.0,2.593616,724
2,3,52.0,7.442820,762,67.0,3.063241,460,29.0,3.200883,302
3,4,58.0,8.343869,452,74.0,3.363022,296,29.0,3.205128,156
4,5,68.0,9.669582,294,82.0,3.740757,209,32.0,3.529412,85
5,6,70.0,9.941090,194,87.0,3.934871,134,32.0,3.518519,60
6,7,76.0,10.840932,141,89.0,4.064685,104,38.0,4.204204,37
7,8,75.0,10.781671,106,89.0,4.034091,80,35.0,3.846154,26
8,9,78.0,11.071429,80,90.0,4.112554,63,29.0,3.267974,17
9,10,79.0,11.300640,67,91.0,4.124579,54,31.0,3.418803,13


In [146]:
# For intersections with a specific complaint type:                                                                                                                 
#    what percentage had a crash?                                                                                                                                   
requests = {}
for data in concern_data:
    if data['REQUESTTYPE'] not in requests.keys():
        requests[data['REQUESTTYPE']] = 1

requests = {}
all_unique = []

for k, v in concerns.iteritems():
    unique_requests = {}
    for request in v['REQUESTTYPE']:
        # Clean up badly formatted request types                                                                                                                    
        vals = request.split('nbsp;')
        if len(vals) > 1:
            request = vals[1]

        if request not in unique_requests.keys():
            unique_requests[request] = 0
        unique_requests[request] += 1

        if request not in requests.keys():
            requests[request] = {
                'crashes': 0,
                'total': 0,
                'inter_crashes': 0,
                'inter_total': 0,
                'non_inter_crashes': 0,
                'non_inter_total': 0,
                'ped_inter_crashes': 0,
                'ped_non_inter_crashes': 0
            }
        if str(k) in crashes.keys():
            requests[request]['crashes'] += 1
        requests[request]['total'] += 1
        if is_inter(k):
            if str(k) in crashes.keys():
                requests[request]['inter_crashes'] += 1
                if crashes[k]['ped']:
                    requests[request]['ped_inter_crashes'] += 1
            requests[request]['inter_total'] += 1
        else:
            if str(k) in crashes.keys():
                requests[request]['non_inter_crashes'] += 1
                if crashes[k]['ped']:
                    requests[request]['ped_non_inter_crashes'] += 1
            requests[request]['non_inter_total'] += 1


    for key, value in unique_requests.iteritems():
        if value > 1:
            all_unique.append([key, value])

by_type = {}
for k, v in all_unique:
    if k not in by_type.keys():
        by_type[k] = 0
    by_type[k] += 1

results = []
for k, v in requests.iteritems():
    if requests[k]['total'] >= 100:
        results.append([
            k,
            by_type[k],
            round(100 * float(requests[k]['crashes'])/float(requests[k]['total'])),
            requests[k]['crashes'],
            requests[k]['total'],
            round(100 * float(requests[k]['inter_crashes'])/float(requests[k]['inter_total'])),
            requests[k]['inter_crashes'],
            requests[k]['inter_total'],
            round(100 * float(requests[k]['non_inter_crashes'])/float(requests[k]['non_inter_total']) if requests[k]['non_inter_total'] else 0),
            requests[k]['non_inter_crashes'],
            requests[k]['non_inter_total'],
            round(100 * float(requests[k]['ped_inter_crashes'])/float(requests[k]['inter_total'])),
            requests[k]['ped_inter_crashes'],
            requests[k]['ped_non_inter_crashes']
        ])
tot_ped = 0
tot_non_ped = 0
for k,v in crashes.iteritems():
    if v['ped'] is True:
        tot_ped += 1
    else:
        tot_non_ped += 1
print tot_ped
print tot_non_ped

df = pd.DataFrame(results, columns=[
    'Request type',
    '# of requests of this type that appear more than once at a segment',
    '% of segments with this type of request with a crash',
    '# of segments with this type of request with a crash',
    '# of segments with this type of a request total',
    '% of intersections with this type of request with a crash',
    '# of intersections with this type of request with a crash',
    '# of intersections with this type of request total',
    '% of non-intersections with this type of request with a crash',
    '# of non-intersections with this type of request with a crash',
    '# of non-intersections with this type of request total',
    '% of intersections with this type of request with a ped crash',
    '# of ped crashes in intersection with this request',
    '# of ped crashes in non-intersection with this request'
])
df.sort_values(by="# of segments with this type of a request total", ascending=0)


976
2454


,Request type,# of requests of this type that appear more than once at a segment,% of segments with this type of request with a crash,# of segments with this type of request with a crash,# of segments with this type of a request total,% of intersections with this type of request with a crash,# of intersections with this type of request with a crash,# of intersections with this type of request total,% of non-intersections with this type of request with a crash,# of non-intersections with this type of request with a crash,# of non-intersections with this type of request total,% of intersections with this type of request with a ped crash,# of ped crashes in intersection with this request,# of ped crashes in non-intersection with this request
14,of something that is not listed here,183,44.0,633,1429,61.0,525,858,19.0,108,571,32.0,277,33
0,people speed,118,26.0,319,1226,39.0,152,385,20.0,167,841,16.0,63,36
4,bike facilities don't exist or need improvement,88,34.0,389,1136,61.0,208,341,23.0,181,795,28.0,95,38
11,people run red lights / stop signs,112,50.0,396,788,56.0,372,670,20.0,24,118,29.0,192,6
12,people don't yield while turning,89,63.0,406,644,70.0,391,561,18.0,15,83,40.0,225,3
1,people double park their vehicles,83,35.0,221,631,54.0,98,180,27.0,123,451,32.0,58,36
5,it’s hard to see / low visibility,28,41.0,178,429,49.0,151,309,23.0,27,120,18.0,57,6
7,people don't yield while going straight,37,47.0,168,361,54.0,145,267,24.0,23,94,27.0,71,11
10,sidewalks/ramps don't exist or need improvement,34,36.0,118,329,54.0,79,147,21.0,39,182,20.0,29,13
13,"the wait for the ""Walk"" signal is too long",49,68.0,204,301,76.0,196,259,19.0,8,42,45.0,116,3


In [40]:
# Pedestrian vs. bike vs. car
# Compare counts against ATR/TMC volume counts
# Parsing extra field's text?